In [12]:
import os
import pandas as pd
import re
import zipfile

In [13]:
fileList = os.listdir()

In [14]:
ref = pd.read_excel("VIP-5new.xlsx")

In [15]:
summaryFiles=''
sampleIndex = {}
errorResults = []
result = pd.DataFrame(columns=["正确编辑占总体的比例",
                               "正确编辑占总编辑的比例",
                               "Indel占总体比例",
                             
                               "NHEJ占总体体比例",
                               "总读数",
                               "实际使用读数",
                               ])

In [16]:
n = 0
for i in ref.index:
    name = ref.loc[i,"Unnamed: 0"].strip()
    resultDir = "results/"+name
    for f in os.listdir(resultDir):
        if "html" in f:
            pass
        elif "ipynb" in f:
            pass
        else:
            folder = f
            logFile = resultDir + "/" + folder + "/CRISPResso_RUNNING_LOG.txt"
            allelesFrequencyTable = resultDir + "/" + folder + "/Alleles_frequency_table.zip"
            resultFile = resultDir + "/" +  folder + "/CRISPResso_quantification_of_editing_frequency.txt"
    
    
    #判断有没有对这个样品进行分析
    if os.path.isfile(logFile):
        #分析日志存在
        pass
    else:
        #分析日志不存在
        result.loc[name,"正确编辑占总体的比例"] = "No enough reads"
        print(name + "error")
        continue #跳过循环，下一个样品
        
    #日志存在，继续进行分析。打开日志文件，读取信息
    with open(logFile) as log:
        o = log.read()
        if "ERROR" in o: #分析过程中出错，一般为reads数为0才出错。其次是分析窗口超出范围。
            errorResults.append(name)
            result.loc[name,"正确编辑占总体的比例"] = "No enough reads"
            print(name + " error")
            continue
        else:
            # 1. 解压、读取分析结果文件
            allelesFrequencyTableZip =zipfile.ZipFile(allelesFrequencyTable)
            allelesData = allelesFrequencyTableZip.read('Alleles_frequency_table.txt')
            allelesFrequencyTableZip.extractall(resultDir + "/" +  folder + "/")
            
            # 2. 逐条进行判断是否为ambiguous Indel
            ambiguousIndel = 0
            allelesFrequencyTable = pd.read_csv( resultDir + "/" + folder + "/Alleles_frequency_table.txt",sep='\t')
            for i in allelesFrequencyTable.index:
                if allelesFrequencyTable.loc[i,"Reference_Name"] == "AMBIGUOUS_Reference":
                    if allelesFrequencyTable.loc[i,"n_deleted"] + allelesFrequencyTable.loc[i,"n_deleted"] > 0:
                        reads = allelesFrequencyTable.loc[i,"#Reads"]
                        ambiguousIndel = ambiguousIndel + reads
            
            # 3. 读取非ambiguous 序列的indel数据
            resultFrame = pd.read_csv(resultFile,sep='\t')
            #print(ambiguousIndel)
            HDR_unmodified = resultFrame.loc[1,'Unmodified']
            allHDR = resultFrame.loc[1,'Reads_aligned']
            HDR_modified = resultFrame.loc[1,'Modified']
            reads_aligned = resultFrame.loc[1,'Reads_aligned_all_amplicons']
            reads = resultFrame.loc[1,'Reads_in_input']
            NHEJreads = resultFrame.loc[0,"Modified"]
            insertion = int(resultFrame.loc[0,"Insertions"]) + int(resultFrame.loc[1,"Insertions"])
            deletion = int(resultFrame.loc[0,"Deletions"]) + int(resultFrame.loc[1,"Deletions"])
            insertionAndDeletion = int(resultFrame.loc[0,"Insertions and Deletions"]) + int(resultFrame.loc[1,"Insertions and Deletions"])
            
            substitution = int(resultFrame.loc[0,"Substitutions"]) + int(resultFrame.loc[1,"Substitutions"])
            HDR_insertion = resultFrame.loc[0,"Insertions"]
            
            
            # 4. 测序深度过滤
            if reads_aligned < 1500:
                result.loc[name,"正确编辑占总体的比例"] = "No enough reads"
                n = n + 1
                continue
                
            
            result.loc[name,"总读数"] = reads
            result.loc[name,"实际使用读数"] = reads_aligned
            
            try:
                result.loc[name,"正确编辑占总体的比例"] = "%.2f%%" %  (float(HDR_unmodified)/float(reads_aligned) *100)
            except:
                result.loc[name,"正确编辑占总体的比例"] = "%.2f%%" % (float(0))
                        
            try:
                result.loc[name,"正确编辑占总编辑的比例"] = "%.2f%%" %(float(HDR_unmodified)/float(allHDR)*100)
            except:
                result.loc[name,"正确编辑占总编辑的比例"] = "%.2f%%" %(float(0)*100)
                    
            try:
                result.loc[name,"NHEJ占总体体比例"] = "%.2f%%" % (float(NHEJreads)/float(reads_aligned)*100)
            except:
                result.loc[name,"NHEJ占总体体比例"] = "%.2f%%" % (float(0)*100)
                        
            #try:
            #    result.loc[name,"Indel占总体比例"] = "%.2f%%" % ((float(insertion) + float(deletion)) / float(reads_aligned)*100)
            #except:
            #    result.loc[name,"Indel占总体比例"] = "%.2f%%" % (float(0)*100)
                
            try:
                result.loc[name,"Indel占总体比例"] = "%.2f%%" % ((float(insertion) + float(deletion) + float(ambiguousIndel) - insertionAndDeletion)  / float(reads_aligned)*100)
            except:
                result.loc[name,"Indel占总体比例"] = "%.2f%%" % (float(0)*100)
            

20220213-1 error
20220213-4 error
20220213-13 error
20220213-24 error
20220219-52 error
20220225-36 error
20220225-41 error
20220225-42 error
20220225-43 error
20220225-44 error
20220623-31 error


In [19]:
result.loc["备注","正确编辑占总体的比例"] = "有发生insertion 或者 deletetion的，或者两者同时发生的算为一个indel。统计来源：与原始amplicon类似的reads（NHEJ），与预期序列类似的（imperfect HDR），与原始、预期都相似但无法判断的reads（AMBIGUOUS）"
result.loc["Method","正确编辑占总体的比例"] = "Modified genome was amplified and sequenced using illumina MiniSeq®. Each amlicon-seq data was analysed wiht CRISPResso2 and summarized by a home-made script. The indel is regard as the reads with insertion or deletion, but subsitution. Data analysis was performed by Qiqin Mo, and the code is avalibele "

In [9]:
import sys

In [11]:
os.path.isfile("/home/d")

False

In [8]:
result.head(30)

,正确编辑占总体的比例,正确编辑占总编辑的比例,Indel占总体比例,NHEJ占总体体比例,总读数,实际使用读数
20220213-1,0.02%,100.00%,0.06%,12.71%,12960,12555
20220213-2,59.38%,87.94%,2.64%,8.16%,28322,18886
20220213-3,48.20%,88.58%,3.27%,6.60%,37178,25599
20220213-4,No enough reads,NaN,NaN,NaN,NaN,NaN
20220213-5,0.01%,100.00%,0.06%,13.72%,15457,15102
20220213-6,49.72%,89.74%,1.66%,6.14%,32063,20334
20220213-7,0.02%,92.86%,0.06%,13.66%,65717,64305
20220213-8,59.89%,89.08%,2.22%,7.00%,37932,24026
20220213-9,0.02%,100.00%,0.05%,6.58%,13474,13167
20220213-10,0.01%,80.00%,0.45%,8.07%,45411,44503


In [21]:
result.to_excel("结果汇总-1500.xlsx")